## Examining the Therapeutic Touch

Welcome to Lab 7!

After such an extensive introduction to programming for data science, we are finally moving into the section of the course where we can apply our new skils to answer real questions.  

In this lab, we'll use testing techniques that were introduced in lecture to test the idea of the therapeutic touch, the idea that some practictioners can feel and massage your human energy field. 

But first, let us learn what _confidence intervals_ are, how to compute them, and how they can help with testing our hypotheses.

In [ ]:
# Run this cell, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
from matplotlib import patches
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

# These lines load the tests.
from client.api.notebook import Notebook
#ok = Notebook('lab07.ok')
_ = ok.auth(inline=True)

## Skittles' Color Distribution

Skittles, is a colorful candy manufactured by the Wrigley company. The original flavors are Orange, Lemon, Lime, Grape and Strawberry. You expect that in each bag of Skittles there is almost an equal proportion of each flavor. However, Sierra is convinced that there are way more green Skittles than there are supposed to be. (She is not very happy that the original lime flavor got replaced with the green apple.) [Commenters on Reddit](https://www.reddit.com/r/dataisbeautiful/comments/5bmo3a) suspect that there are too many yellow Skittles in each party-sized bag. Who is right?

To test her hypothesis, Sierra bought a party-sized bag of Skittles and recruited her housemate to help her count each color. In 5 minutes, 27 seconds they had the distribution of colors in the bag!

Strawberry(red): 51
Orange (orange): 65
Green apple (green): 70
Lemon (yellow): 94
Grape (purple): 103
Total: 383

In [ ]:
skittles_bag = make_array(51, 65, 70, 94, 103)
skittles_bag_total = skittles_bag.sum()
skittles_bag_total

In [ ]:
skittles_bag_proportions = skittles_bag / skittles_bag_total
skittles_bag_proportions

In [ ]:
# Make an array that represents our model's distribution of flavors
# i.e., a bag should have an equal proportion of each flavor

model_bag_proportions = make_array(1,1,1,1,1) * ...
model_bag_proportions

In [ ]:
skittles = Table().with_columns(
    'Color', make_array('Orange', 'Lemon', 'Apple', 'Grape', 'Strawberry'),
    'Bag', skittles_bag_proportions,
    'Model', model_bag_model
)

skittles

In [ ]:
#  A bar chart is helpful for visualizing the differences.
skittles.barh('Color')

### Comparison with Bags Selected at Random

What if we select a random sample of the same size as `skittles_bag_total` from the population of possibe Skittles flavors? Will the distribution of their colors/flavors look like the distribution of the bag that Sierra bought?

We can answer these questions by using `sample_proportions` function and augmenting the `skittles` table with a column of the proportions in our _one_ sample. That is, we will "put together" a virtual bag of Skittles, using proportions in our model, and call it our sample bag.

Let's first look at how `sample_proportions` function works. 

### `sample_proportions` function

The `sample_proportions` function in the `datascience` library takes two arguments:

* the sample size
* the distribution of the categories in the population, as a list or array of proportions that add up to 1

It returns an array containing the distribution of the categories in a random sample of the given size taken from the population. That’s an array consisting of the sample proportions in all the different categories.

To see how to use this, remember that according to our model, the bag is selected at random from a population of possible Skittles colors is supposed to have roughly the same number of each color. Thus the distribution of the five categories was represented as an array, which we have assigned to the name `model_bag_proportions`. 

Now let’s sample at random times from this distribution, _with replacement_, and see what proportions of the five categories we get in our sample. 

**What should be our sample size?**

In [ ]:
model_bag_proportions
sample_bag = sample_proportions(..., model_bag_proportions)
sample_bag

In [ ]:
# Let's create a table to hold the result of our random sample
sample_table = Table().with_columns(
    'Color', make_array('Orange', 'Lemon', 'Apple', 'Grape', 'Strawberry'),
    'Sample', sample_bag
)
    
#  A bar chart shows the distibution we get in our random sample
sample_table.barh('Color')

In [ ]:
# Let's add the random sample to our main table to compaer the differences
skittles_and_sample = skittles.with_column('Random Sample', sample_bag)
skittles_and_sample

In [ ]:
#  A bar chart is helpful for visualizing the differences.
skittles_and_sample.barh('Color')

The bar chart shows that the distribution of the random sample better resembles the possible Skittles flavors but the distribution of the bag does not.

### The Distance between Two Distributions

Now we have to quantify the distance between two distributions. For example, we can measure the distance between the blue and gold distributions, like we did in the lecture. That would give us a quantity called the _total variation distance_ (TVD) between two distributions. Since we are going to be computing total variation distance repeatedly, we will write a function to compute it.

The function `total_variation_distance` returns the TVD between distributions in two arrays.

In [ ]:
def total_variation_distance(distribution_1, distribution_2):
    return sum(np.abs(distribution_1 - distribution_2)) / 2

Let’s check that it gives the right answer when we use it to compute the distance between the blue (Sierra's bag) and gold (model, possible Skittles) distributions above.

In [ ]:
bag_tvd = total_variation_distance(skittles.column('Bag'), skittles.column('Model'))
bag_tvd

In the cell below we use the function to compute the TVD between the distributions of the possible flavors and one random sample. This is the code for simulating *one* value of our statistic. Recall that `model_bag_proportions` is the array containing the distribution of the possible flavors/colors.

In [ ]:
sample_bag = sample_proportions(383, model_bag_proportions)
total_variation_distance(sample_bag, model_bag_proportions)

**How does the values of this statistic compare to the one we got when we compared Sierra's bag to the model distribution?**

Write down your observation.

### Predicting the Statistic Under the Model of Random Selection

The total variation distance between the distributions of the random sample and the possible Skittles colors is the statistic that we are using to measure the distance between the two distributions. By repeating the process of sampling, we can see how much the statistic varies across different random samples.

The code below simulates the statistic based on a large number of replications of the random sampling process, following our usual sequence of steps for simulation. The body of the `for` loop repeats the code we used to simulate one value of the statistics, and then appends the simulated value to the collection array `tvds`.

In [ ]:
# Simulate total variation distance between
# distribution of sample selected at random
# and distribution of possible flavors

model_bag_proportions = skittles.column('Model')
bag_size = 383

tvds = make_array()

repetitions = 5000
for i in np.arange(repetitions):
    sample_bag_distribution = sample_proportions(bag_size, model_bag_proportions)
    new_tvd = total_variation_distance(sample_bag_distribution, model_bag_proportions)
    tvds = np.append(tvds, new_tvd)

The empirical histogram of the simulated distances shows that drawing 383 Skittles at random from the pool of possible colors results in a distribution that rarely deviates from the possible Skittles' color distribution by more than about 0.1. We can compare this to the observed sample statistic, which is the TVD that we computed for Sierra's bag. The observed value is shown as a red dot on the horizontal axis. The last line of code just draws the dot.

In [ ]:
Table().with_column('TVD', tvds).hist(bins=np.arange(0, 0.2, 0.005))
plt.scatter(bag_tvd, 0, color='red');

### Assessing the Model of Random Selection

Is the observed statistic “too high” in relation to this distribution? In other words, is the red far enough out into the right hand tail of the histogram for you to think that it is “too far”?

It’s up to you to decide! Use your judgment.

### Conventional Cut-offs and the P-value

If you don’t want to make your own judgment, there are conventions that you can follow. These conventions tell us how far out into the tails of the histogram is considered “too far”.

The conventions are based on the area in the tail, starting at the observed statistic (the red dot) and looking in the direction that makes us lean toward the alternative hypothesis. In the beginning, we stated that our model expects that in each bag of Skittles there is almost an equal proportion of each flavor, which means that each color has the same probability of being selected.

If that's our Null Hypothesis we can state is as follows:

**Null Hypothesis** ($H_0$): On average, the proportion of each Skittles color is 0.2.

**Alternative Hypothesis**: 

Write down the Alternative Hypothesis.


Let's see if we can compute how many samples had either the same TVD or below.

Remember that in a histogram, area represents percent. To find the area in the tail, we have to find the percent of sample values that were ... than the sample statistic (`bag_tvd`, TVD of the bag), where the red dot is. The array `tvds` contains the results for all repetitions of the random sampling, and `bag_tvd` is the value of Sierra's sample.

In [ ]:
np.count_nonzero(tvds ... bag_tvd)/repetitions

That means that if we had drawn the bags at random from all Skittles bags, this is the chance that their difference would be the ... than Sierras.

This chance has an impressive name. It is called the **observed significance level** of the test. That’s a mouthful, and so it is commonly called the **P-value** of the test.

### Definition of P-value

The P-value is the chance, based on the model in the null hypothesis, that the test statistic is equal to the value that was observed in the data or is even further in the direction of the alternative.

If a P-value is small, that means the tail beyond the observed statistic is small and so the observed statistic is far away from what the null predicts. This implies that the data support the alternative hypothesis better than they support the null.

How small is “small”? According to the conventions:

> If the P-value is less than 5%, it is considered small and the result is called “statistically significant.”

> If the P-value is even smaller – less than 1% – the result is called “highly statistically significant.”

Is the result of Sierra's test statistically significant?

When you make a conclusion in this way, we recommend that you don’t just say whether or not the result is statistically significant. Along with your conclusion, provide the observed statistic and the P-value as well, so that readers can use their own judgment.



Now, let's look at a different experiment and see if we can apply what we learned to this new scenario.

# The Therapeutic Touch

The Therapeutic Touch (TT) is the idea that everyone can feel the Human Energy Field (HEF) around individuals. Certain practictioners claim they have the ability to feel the HEF and can massage it in order to promote health and relaxation in individuals. Those who practice TT have described different people's HEFs as "warm as Jell-O" and "tactile as taffy". 

TT was a popular technique used throughout the 20th century that was touted to be a great way to bring balance to a person's health. 

### Emily Rosa

Emily Rosa was a 4th grade student who had wide exposure to the world of TT due to her parents. Her parents were both medical practitioners and skeptics of the idea of TT. 

For her 4th grade science fair project, Emily decided to test whether or not TT practitioners could truly have any interaction with a person's HEF. 

**Question 1:** Discuss with the individuals around you how you would set up an experiment to test this.

*Write your answer here, replacing this text.*

### Emily's Experiment

Emily's experiement was clean, simple, elegant, and effective. Due to her parents' occupations in the medical field, she had wide access to people who claimed to be TT practitioners. 

Emily took 21 TT practitioners and used them for her science experiment. She would take a TT practitioner, ask them to extend their hands through a screen (through which they couldn't see). On the other side would be Emily, who would flip a coin and put out either her left hand or her right hand, depending on the coin. The TT practitioner would then have to correctly answer which hand Emily put out. Overall, through 210 samples, the practitioner picked the correct hand 44% of the time. 

Emily's main goal here was to test whether or not the TT practicioners guesses were random, like a flip of the coin. In most medical experiments, this is the norm. We want to test whether or not the treatment has an effect, *not* whether or not the treatment actually works. 

We will now begin to formulate this experiment in terms of the terminology we learned in this course. 

**Question 2**: What are the null and alternative hypothesis for Emily's experiment? Discuss with students around you to come to a conclusion. 

**Your Answer Here:**

Null Hypothesis: 

Alternative Hypothesis: 

**Question 3:** Remember that the pracitioner got the correct answer 44% of the time. According to the null hypothesis, on average, what proportion of times do we expect the practitioner to guess the correct hand? Make sure your answer is between 0 and 1. 

In [ ]:
expected_correct = ...
expected_correct

In [ ]:
_ = ok.grade('q3')

The goal now is to see if our deviation from this expected proportion of correct answers is due to something other than chance. 

**Question 4:** What is a valid test statistic we can use to test our model? Assign `valid_ts` to a list of integers representing the following options: 

1. The difference of the expected percent correct and the actual percent correct
2. The absolute difference of the expected percent correct and the actual percent correct
3. The sum of the expected percent correct and the actual percent correct

There may be more than one correct answer. 

In [ ]:
valid_ts = ...
valid_ts

In [ ]:
_ = ok.grade('q4')

**Question 5:** Define the function `test_statistic` which takes in an expected proportion and an actual proportion, and returns the value of the test statistic chosen above. Assume that you are taking in _proportions_, but you want to return your answer as a _percentage_. 

In [ ]:
def test_statistic(expected_prop, actual_prop):
    ...


In [ ]:
_ = ok.grade('q5')

**Question 6:** Use your newly defined function to calculate the observed test statistic from Emily's experiment. 

In [ ]:
observed_test_statistic = ...
observed_test_statistic

In [ ]:
_ = ok.grade('q6')

Is this a test statistic that is likely if the null hypothesis was true or is the deviation from the expected proportion due to something other than chance?

In order to answer this question, we must simulate the experiment _as though the null hypothesis was true_, and calculate the test statistic per each simulation. 

**Question 7:** To begin simulating, we start by creating an array that has two items in it. The first item should be the proportion of times, assuming the null hypothesis/model is true, i.e., a TT practictioner picks the correct hand. The second item should be the proportion of times, under the same assumption, that the TT practicioner picks the incorrect hand. Assign `model_proportions` to this array. 

Then, using the `sample_proportions` function, simulate Emily running through this experiment 210 times (as was done in real life), and assign the proportion of correct answers to `simulation_proportion`.

Lastly, define `one_test_statistic` to the test statistic of this one simulation. 

In [ ]:
model_proportions = ...
simulation_proportion = ...
one_test_statistic = ...
one_test_statistic

In [ ]:
_ = ok.grade('q7')

**Question 8:** Let's now see what the distribution of test statistics is actually like under our fully specified model. Assign `simulated_test_statistics` to an array of 1000 test statistics that you simulated assuming the null hypothesis is true. 

*Hint:* This should follow the same pattern as normal simulations, in combination with the code you did in the previous problem.  

In [ ]:
num_repetitions = 1000
num_guesses = 210

simulated_test_statistics = ...

for ... in ...:
    ...


In [ ]:
_ = ok.grade('q8')

Let's view the distribution of the simulated test statistics under the null, and visually compare how the observed test statistic lies against the rest. 

In [ ]:
t = Table().with_column('Simulated Test Statistics', simulated_test_statistics)
t.hist()
plt.scatter(observed_test_statistic, 0, color='red', s=30)

We can make a visual argument as to whether or not we believe the observed test statistic is likely to occur under the null, or we can use the definition of p-values to help us make a more formal argument. 

**Question 9:** Assign `p_value` to the integer corresponding to the correct definition of what a p-value really is. 

1. The chance, under the null hypothesis, that the test statistic is equal to the value that was observed
2. The chance, under the null hypothesis, that the test statistic is equal to the value that was observed or is even further in the direction of the alternative
3. The chance, under the alternative hypothesis, that the test statistic is equal to the value that was observed or is even further in the direction of the null 
4. The number of times, under the null hypothesis, that the test statistic is equal to the value that was observed or is even further in the direction of the alternative

In [ ]:
p_value = ...
p_value

In [ ]:
_ = ok.grade('q9')

**Question 10:** Using the definition above, calculate the p-value of Emily's observed value in this experiment. 

*Hint:* If our test statistic is further in the direction of the alternative, will it be a larger value or a smaller value? 

In [ ]:
emily_p_val = ...
emily_p_val

In [ ]:
_ = ok.grade('q10')

If our p-value is less than or equal to .05, then this is in favor of our alternative and we reject the null hypothesis. Otherwise, we do not have enough evidence against our null hypothesis. Note that this does **not** say we side in favor with the null hypothesis and accept it, but rather, that we **fail to reject it**.

This should help you make your own conclusions about Emily Rosa's experiment. 

Therapeutic touch fell out of use after this experiment, which was eventually accepted into one of the premier medicine journals that every professional strives to publish in: Emily Rosa (at age nine) was the youngest person to have a research paper published in the Journal of the American Medical Association in 1998. TT practitioners hit back and accused Emily and her family of tampering with the results, while some claimed that Emily's bad spiritual mood towards therapeutic touch made it difficult to read her HEF. Whatever it may be, Emily's experiment is a classic example of how anyone, with the right resources, can test anything they want!

Think to yourself and consult with the mentors about the following questions: 

1. Do we reject the null hypothesis, or fail to reject it? 
2. What does this mean in terms of Emily's experiment? Do the TT practitioners answers follow an even chance model or is there something else at play? 

Lastly, make sure to run all the tests (the next cell has a shortcut for that), save your work, and run the submit cell before you leave lab to ensure your assignment gets graded! 

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

In [ ]:
_ = ok.submit()